In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load data
df = pd.read_csv('../Data/combined_data.csv')
df = df[df['speed'] == 1]
df = df[df['node-id'] == 562]
df = df[df['weight'] != 85.1]
df = df.reset_index(drop=True)

# Classify weights into 3 classes
num_classes = 3
df['weight_class'] = pd.qcut(df['weight'], q=num_classes, labels=False)

# Group by weight_class
weights_by_class = df.groupby('weight_class')['weight'].unique()
print(weights_by_class)

# Parameters
n_time_steps = 400
step = 200

# Feature extraction
features = []
labels = []

for i in range(0, df.shape[0] - n_time_steps, step):
    segment = df.iloc[i: i + n_time_steps]
    xs = segment['ax'].values
    ys = segment['ay'].values
    zs = segment['az'].values
    
    feature = [
        xs.mean(), xs.std(), xs.min(), xs.max(),
        ys.mean(), ys.std(), ys.min(), ys.max(),
        zs.mean(), zs.std(), zs.min(), zs.max()
    ]
    label = segment['weight_class'].mode()[0]
    features.append(feature)
    labels.append(label)

features = np.array(features)
labels = np.array(labels)

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

# SVM Model
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=1.0, random_state=42))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print("SVM Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))

# Decision Tree Model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt))
print("Decision Tree Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))

# KNN Model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy:", accuracy_knn)
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print("KNN Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_knn))


weight_class
0       [86.7, 88.35, 89.4, 90.6, 91.95]
1        [93.35, 94.5, 95.0, 96.1, 97.6]
2    [98.85, 100.0, 101.6, 103.0, 104.5]
Name: weight, dtype: object
Features shape: (4498, 12)
Labels shape: (4498,)
Training set shape: (3598, 12)
Test set shape: (900, 12)
SVM Accuracy: 0.7644444444444445
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87       320
           1       0.79      0.48      0.60       306
           2       0.67      0.93      0.78       274

    accuracy                           0.76       900
   macro avg       0.77      0.77      0.75       900
weighted avg       0.78      0.76      0.75       900

SVM Confusion Matrix:
[[285  23  12]
 [ 47 147 112]
 [  3  15 256]]
Decision Tree Accuracy: 0.7866666666666666
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       320
           1       0.69      0.6